In [1]:
import sys
sys.path.append('/home/mz1482/project/vt-bayesian-opt-bopt_debug/')
import pandas as pd
import numpy as np
from data_analysis import *
from graph import *
from BayesOptLib.bayes_opt.bayesian_optimization import BayesianOptimization
from RandomSampler import RandomSampler
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics.pairwise import euclidean_distances
matplotlib.use('Qt5Agg')  # or can use 'TkAgg', whatever you have/prefer
from prettytable import PrettyTable

def black_box(x, y, z):
    """
    Represents a black box function to maximize for CC of two ECGs given an XYZ coordinate
    :param x:
    :param y: coordinates of the prediction
    :param z:
    :return: CC of that point and the target
    """
    sample_ecg = ecgs[get_index(np.array([x, y, z]),labels)]
    return abs(correlation_coef(target_ecg, sample_ecg))


def black_box_12_lead(x, y, z):
    """
    Represents a black box function to maximize for CC of two ECGs given an XYZ coordinate
    :param x:
    :param y: coordinates of the prediction
    :param z:
    :return: CC of that point and the target
    """
    sample_ecg = ecgs[get_index(np.array([x, y, z]),labels)]
    sample_ecg = np.reshape(sample_ecg, [12, -1])
    nums = 0
    for i in range(12):
#         ccs = np.corrcoef(sample_ecg[i], target_ecg.reshape(12,-1)[i])[0, 1]
        ccs = abs(correlation_coef(target_ecg.reshape(12,-1)[i], sample_ecg[i]))
        if ccs > .9:
            nums += 1

#     return True if nums == 12 else False 
    return nums

In [2]:
data_path = "/home/mz1482/project/vt-bayesian-opt-bopt_debug/data/simu_data_4000/"
# Reading in the ECGs and labels
# aucs = pd.read_csv(data_path+"Heart1/Heart1_AUCS_4000.csv", header=None).to_numpy()
ecgs = pd.read_csv(data_path+"Heart1/Heart1_SimuData_4000_200Cropped.csv", header=None).to_numpy()
labels = pd.read_csv(data_path+"Heart1/Coord1_4000.csv", header=None).to_numpy() / 1000
uvc = pd.read_csv(data_path+"UVC/Coord1_UVC_4000.csv", header=None).to_numpy()
bounds = get_heart_bounds(labels) # getting bounds 

# print("Target: ", target)
init = 10 #initial random sample
steps = 10 # total AL step
af = "ucb" 


In [3]:
# labels = uvc[:,0:3]
# bounds = get_heart_bounds(labels) # getting bounds 
# print("Target: ", target_uvc)
for n in range(10):
    tidx = np.random.randint(0, labels.shape[0])    # Pick out a sample to use as a target
    target, target_ecg,target_uvc = labels[tidx], ecgs[tidx], uvc[tidx]
    optimizer = BayesianOptimization(f=black_box,pbounds=bounds,random_state=None, real_set=labels,ecgs = ecgs,target_ecg = target_ecg)
    gp,lead = optimizer.gpfit(init_points=init, n_iter=steps,  acq=af, kappa = 2.5,kappa_decay=0.90,kappa_decay_delay=5,xi=0)
    #     heart(target,labels)
    max_cc,best, al,pass_lead,loc = result_metric(optimizer,target,target_ecg,labels,ecgs,lead_condition = True)
    f= open("result.txt","a")
    f.write(str(n)+","+str(target[0]) + "," + str(target[1]) + "," + str(target[2])+","+str(best[0]) + "," + str(best[1]) + "," + str(best[2])+","+str(max_cc)+","+str(al)+","+str(pass_lead)+","+str(loc) +"\n")
    f.close()



|   iter    |  target   |     x     |     y     |     z     |
-------------------------------------------------------------
|  1        |  0.5432   |  21.64    | -79.81    |  23.28    |
passing lead 1
|  2        |  0.3354   | -22.03    | -86.17    |  36.93    |
passing lead 4
|  3        |  0.1791   |  11.72    | -95.29    |  11.79    |
passing lead 1
|  4        |  0.1131   | -22.29    | -24.51    | -21.91    |
passing lead 0
|  5        |  0.04614  | -1.243    | -81.34    |  45.41    |
passing lead 3
|  6        |  0.3367   | -34.09    | -84.87    |  41.49    |
passing lead 1
|  7        |  0.2711   | -3.175    | -82.05    |  7.301    |
passing lead 0
|  8        |  0.1897   |  38.42    | -86.15    | -11.74    |
passing lead 0
|  9        |  0.5677   | -32.39    | -34.67    | -27.78    |
passing lead 2
|  10       |  0.2369   | -23.78    | -50.73    |  24.85    |
passing lead 0
|  11       |  0.5371   |  21.46    | -76.77    |  22.33    |
passing lead 1
|  12       |  0.5222   | -31

|  12       |  0.9774   |  28.24    | -60.37    | -17.62    |
passing lead 12
al step 2
lead condition is given
best value [ 28.2402 -60.3659 -17.6187]
max cc is 0.9773907987930367
|   iter    |  target   |     x     |     y     |     z     |
-------------------------------------------------------------
|  1        |  0.313    | -22.29    | -24.51    | -21.91    |
passing lead 0
|  2        |  0.02617  |  27.06    | -76.35    | -14.67    |
passing lead 0
|  3        |  0.02617  |  27.06    | -76.35    | -14.67    |
passing lead 0
|  4        |  0.3551   |  9.577    | -46.35    |  25.53    |
passing lead 1
|  5        |  0.4506   |  22.49    | -48.01    | -44.16    |
passing lead 0
|  6        |  0.06357  | -33.36    | -70.96    |  39.43    |
passing lead 0
|  7        |  0.03895  | -2.891    | -89.02    |  33.01    |
passing lead 1
|  8        |  0.8017   | -31.8     | -34.42    | -26.54    |
passing lead 6
|  9        |  0.1123   | -27.2     | -60.31    |  38.85    |
passing lead 0
| 

passing lead 0
|  8        |  0.1154   |  18.23    | -77.52    | -6.07     |
passing lead 0
|  9        |  0.4813   |  26.7     | -94.3     |  9.058    |
passing lead 2
|  10       |  0.152    |  21.22    | -41.07    | -8.898    |
passing lead 0
|  11       |  0.6644   |  26.45    | -47.84    |  24.16    |
passing lead 3
|  12       |  0.7142   |  20.84    | -50.4     |  31.33    |
passing lead 2
|  13       |  0.6967   |  20.52    | -56.17    |  28.45    |
passing lead 3
|  14       |  0.7339   |  14.56    | -57.97    |  37.97    |
passing lead 3
|  15       |  0.6872   |  21.77    | -62.95    |  27.62    |
passing lead 2
|  16       |  0.7357   |  14.1     | -53.1     |  39.4     |
passing lead 3
|  17       |  0.7289   |  12.48    | -57.7     |  38.35    |
passing lead 4
|  18       |  0.6543   |  24.8     | -65.81    |  22.6     |
passing lead 2
|  19       |  0.6392   |  28.85    | -77.64    |  21.5     |
passing lead 2
|  20       |  0.676    |  19.14    | -56.57    |  26.77    |

In [4]:
optimizer._space._target[iteration+init_points]

NameError: name 'iteration' is not defined

In [ ]:
optimizer._space.target_func(best[0],best[1],best[2])

In [ ]:
optimizer._space.probe(optimizer.res[12]['params'])

In [ ]:
optimizer.max

In [ ]:
t = np.array(optimizer.res)
t[[1]][0]

In [ ]:
t = np.fromiter(optimizer.x_best.values(),dtype = float)
# r = t['params']
t

In [ ]:
a=np.fromiter(r.values(),dtype=float)
a.shape

In [ ]:
init = len(optimizer.visited)-len(optimizer.predicted)
Y = optimizer.lead_num
X = optimizer.visited
max_lead = np.amax(Y)
for i in range(len(Y)):
    if Y[i]==max_lead:
        idx = i
        break
print("best value",np.fromiter(optimizer.res[i]['params'].values(),dtype = float))

In [ ]:


def result_table_xyz(target,optimizer,uvc=False):
    '''
    for 1 experiment it prints/return the table with distance from target at every iteration.
    This function is for xyz coordinates
    '''
    print("The target location (in 3d) is:",target)
    if uvc==False:
        visited = np.asarray(optimizer.visited)
        f = optimizer._space.target
        nn_dis = np.empty((0,1))
        for i in range(len(visited)):
            d = np.sqrt(np.sum((target - visited[i])**2))
            nn_dis = np.append(nn_dis,d)
        nn_dis = nn_dis.reshape(-1,1)
        nn_dis = np.around(nn_dis,2)
        f = f.reshape(-1,1).astype(int)
        it = (np.arange(len(visited))+1).reshape(-1,1).astype(int)
        floating_part = np.around(np.concatenate((visited,nn_dis), axis = 1),2)
        int_part = np.concatenate((it,f), axis = 1)
        result = np.concatenate((int_part,floating_part), axis = 1)
        table = PrettyTable(result.dtype.names)
        table.field_names = ['iteration','passing_lead', 'x','y','z','dis(mm) from target']
        for row in result:
            table.add_row(row)
        return table
    
    

In [ ]:
result_table_xyz(target,optimizer)

In [ ]:
table.title = 'Results for method Foo'
table.field_names = ['Experiment', 'Value']
table.add_row(['bla', 3.14])
table.add_row(['baz', 42.0])

In [ ]:
len(optimizer.predicted)

In [ ]:
f.astype(int)

In [ ]:
it.shape